```mermaid
flowchart TD
    A([Inicio]) --> B[Lectura de documentos]
    B --> C[Extracción de texto]
    C --> D[División en fragmentos relevantes]
    D --> E[Vectorización con TF-IDF]
    E --> F[Creación o actualización de colección en Qdrant]
    F --> G[Indexación de los vectores dispersos]
    G --> H([Repositorio de conocimiento listo])

    H --> I[Recepción de consulta del usuario]
    I --> J[Vectorización de la consulta con el mismo modelo]
    J --> K[Búsqueda semántica en Qdrant]
    K --> L([Resultados más relevantes])
```

In [1]:
!pip install scikit-learn pypdf pandas
!pip install -U "qdrant-client>=1.8.2"
!pip show qdrant-client


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


### Collection Name :

In [2]:
COLLECTION = "code_knowledge"

### Delete Collection :

In [3]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333", prefer_grpc=False, check_compatibility=False)

if client.collection_exists(COLLECTION):
    client.delete_collection(COLLECTION)
    print(f"Collection '{COLLECTION}' deleted.")
else:
    print(f"Collection '{COLLECTION}' does not exist.")

Collection 'code_knowledge' deleted.


### Load Chunk Methods :

In [4]:
import re

def chunk_text(text: str, chunk_size: int = 300, overlap: int = 50) -> list[str]:
    """
    Divide el texto en chunks semánticos usando expresiones regulares.
    - Respeta límites de oración (., !, ?, ;, :) y saltos de párrafo.
    - Mantiene un tamaño aproximado en palabras.
    - Aplica overlap para conservar contexto entre chunks.
    """

    text = text if text else ""
    text = re.sub(r"\s+", " ", text.strip())
    if not text:
        return []

    paragraphs = re.split(r"\n{2,}", text)
    sentences = []

    for para in paragraphs:
        para = para.strip()
        if not para:
            continue
        parts = re.split(r"(?<=[.!?;:])\s+", para)
        sentences.extend(parts)

    chunks = []
    current_chunk = []
    current_len = 0

    for sent in sentences:
        sent = sent.strip()
        if not sent:
            continue
        words = sent.split()
        sent_len = len(words)

        if current_len + sent_len > chunk_size and current_chunk:
            chunks.append(" ".join(current_chunk).strip())

            if overlap > 0 and current_chunk:
                overlap_words = " ".join(" ".join(current_chunk).split()[-overlap:])
                current_chunk = [overlap_words] if overlap_words else []
                current_len = len(overlap_words.split()) if overlap_words else 0
            else:
                current_chunk = []
                current_len = 0

        current_chunk.append(sent)
        current_len += sent_len

    if current_chunk:
        chunks.append(" ".join(current_chunk).strip())

    return [c for c in chunks if c.strip()]

In [5]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable, Union
import uuid
import pandas as pd
from pypdf import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer

from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    PointStruct, SparseVector, SparseVectorParams,
)
from qdrant_client import models as qm


BATCH_SIZE = 1024
SPARSE_NAME = "text"  

def is_existing_file(p: Union[str, Path]) -> bool:
    try:
        return Path(p).exists()
    except Exception:
        return False

def read_pdf_pages(pdf_path: Path) -> list[dict]:
    r = PdfReader(str(pdf_path))
    out = []
    for i, page in enumerate(r.pages, start=1):
        text = page.extract_text() or ""
        out.append({"page": i, "text": text})
    return out

def chunk_text_old(text: str, chunk_size: int = 300, overlap: int = 50) -> list[str]:
    toks = text.split()
    out, start, n = [], 0, len(toks)
    while start < n:
        end = min(start + chunk_size, n)
        ch = " ".join(toks[start:end]).strip()
        if ch:
            out.append(ch)
        if end == n:
            break
        start = max(0, end - overlap)
    return out

def csr_row_to_sparse(row_csr):
    return row_csr.indices.tolist(), row_csr.data.tolist()

def to_uuid_str(namespace: str, local_id: str) -> str:
    return str(uuid.uuid5(uuid.NAMESPACE_URL, f"{namespace}:{local_id}"))


def build_corpus_from_documents(
    documents: Iterable[Union[str, Path]],
    chunk_size: int = 300,
    overlap: int = 50,
) -> pd.DataFrame:
    """
    Accepts:
      - Path/str to PDF files
      - raw text strings (treated as virtual docs)
    Returns DataFrame: id, source, page, text
    """
    rows = []
    for doc in documents:
        if is_existing_file(doc):
            p = Path(doc)
            if p.suffix.lower() == ".pdf":
                pages = read_pdf_pages(p)
                for pg in pages:
                    for j, c in enumerate(chunk_text(pg["text"], chunk_size, overlap), start=1):
                        rows.append({
                            "id": f"{p.name}:p{pg['page']}_c{j}",
                            "source": str(p),
                            "page": int(pg["page"]),
                            "text": c,
                        })
            else:
                txt = Path(p).read_text(encoding="utf-8", errors="ignore")
                for j, c in enumerate(chunk_text(txt, chunk_size, overlap), start=1):
                    rows.append({
                        "id": f"{p.name}:c{j}",
                        "source": str(p),
                        "page": None,
                        "text": c,
                    })
        else:
            txt = str(doc)
            pseudo = f"raw:{hash(txt) & 0xffffffff:x}"
            for j, c in enumerate(chunk_text(txt, chunk_size, overlap), start=1):
                rows.append({
                    "id": f"{pseudo}:c{j}",
                    "source": pseudo,
                    "page": None,
                    "text": c,
                })

    df = pd.DataFrame(rows)
    if df.empty:
        raise ValueError("No chunks produced. Check your inputs.")
    return df


def create_sparse_collection(client: QdrantClient, collection: str, sparse_name: str = SPARSE_NAME):
    if client.collection_exists(collection):
        client.delete_collection(collection)
    client.create_collection(
        collection_name=collection,
        vectors_config={},  # no dense vectors
        sparse_vectors_config={sparse_name: SparseVectorParams()},
    )

def upsert_dataframe_tfidf_sparse(
    client: QdrantClient,
    collection: str,
    df: pd.DataFrame,
    vectorizer: TfidfVectorizer,
    batch_size: int = BATCH_SIZE,
    sparse_name: str = SPARSE_NAME,
    namespace_for_uuid: str | None = None,
):
    """
    Upserts df rows (id, source, page, text) using TF-IDF vectors into Qdrant.
    Uses UUIDv5 *string* IDs derived from df["id"] (and optional namespace).
    """
    X = vectorizer.transform(df["text"])  # CSR aligned with df
    n = len(df)
    total = (n + batch_size - 1) // batch_size

    for b in range(total):
        lo, hi = b * batch_size, min((b + 1) * batch_size, n)
        points = []
        for i in range(lo, hi):
            local_id = str(df.iloc[i]["id"])  # human-readable
            point_id = to_uuid_str(namespace_for_uuid or collection, local_id)
            src = df.iloc[i]["source"]
            page_raw = df.iloc[i]["page"]  # may be None/NaN/int
            page_val = int(page_raw) if pd.notna(page_raw) else None
            txt = df.iloc[i]["text"]
            idx, vals = csr_row_to_sparse(X[i])

            payload = {"source": src, "chunk_id": local_id, "text": txt}
            if page_val is not None:
                payload["page"] = page_val

            # named sparse vector goes in the "vector" dict
            points.append(
                PointStruct(
                    id=point_id,
                    payload=payload,
                    vector={sparse_name: SparseVector(indices=idx, values=vals)},
                )
            )
        client.upsert(collection_name=collection, points=points)
        print(f"Upserted batch {b+1}/{total} ({hi - lo} points)")


def build_and_index(
    documents: Iterable[Union[str, Path]],
    collection: str = COLLECTION,
    chunk_size: int = BATCH_SIZE,
    overlap: int = 50,
    sparse_name: str = SPARSE_NAME,
):
    # 1) Build corpus
    df = build_corpus_from_documents(documents, chunk_size, overlap)
    print("Corpus shape:", df.shape)

    # 2) Fit TF-IDF on ALL texts (shared vocab)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=True)
    vectorizer.fit(df["text"])

    # 3) Qdrant client (HTTP; modern API)
    client = QdrantClient(
        url="http://localhost:6333",
        prefer_grpc=False,
        # Set True when client/server majors/minors are aligned
        check_compatibility=False,
    )

    # 4) Create collection (only sparse)
    create_sparse_collection(client, collection, sparse_name)

    # 5) Upsert
    upsert_dataframe_tfidf_sparse(
        client=client,
        collection=collection,
        df=df,
        vectorizer=vectorizer,
        batch_size=BATCH_SIZE,
        sparse_name=sparse_name,
        namespace_for_uuid=collection,
    )

    return client, vectorizer, df

# =========================
# Retrieval (use deprecated but stable .search to avoid resolver issues)
# =========================
def query_tfidf(
    client: QdrantClient,
    collection: str,
    vectorizer: TfidfVectorizer,
    query: str,
    top_k: int = 5,
    sparse_name: str = SPARSE_NAME,
    source_filter: str | None = None,
    page_eq: int | None = None,
):
    """
    Uses client.search (deprecated) with top-level models to avoid the fast-embed resolver
    rejecting http.models.* types on your current client version.
    """
    # TF-IDF → sparse indices/values
    q = vectorizer.transform([query])
    idx = q.indices.tolist()
    vals = q.data.tolist()

    # Optional filters
    q_filter = None
    must = []
    if source_filter:
        must.append(qm.FieldCondition(key="source", match=qm.Match(value=source_filter)))
    if page_eq is not None:
        must.append(qm.FieldCondition(key="page", match=qm.Match(value=int(page_eq))))
    if must:
        q_filter = qm.Filter(must=must)

    results = client.search(
        collection_name=collection,
        query_vector=qm.NamedSparseVector(
            name=sparse_name,
            vector=qm.SparseVector(indices=idx, values=vals),
        ),
        query_filter=q_filter,
        limit=top_k,
        with_payload=True,
    )

    out = []
    print(f"\n🔎 '{query}' → top {top_k}\n")
    for p in results:
        pl = p.payload or {}
        out.append({
            "id": str(p.id),
            "score": p.score,
            "source": pl.get("source"),
            "page": pl.get("page"),
            "chunk_id": pl.get("chunk_id"),
            "text": pl.get("text"),
        })
        print(f"• {p.score:.4f} | {pl.get('source')} | p{pl.get('page')} | {pl.get('chunk_id')}")
        print(f"  {(pl.get('text') or '')[:170]}...\n")
    return out

# =========================
# Example usage
# =========================
if __name__ == "__main__":
    docs = [
        Path("./knowledge/cc_knowledge_book.pdf"),
        Path("./knowledge/Fluent.Python.2nd.Edition.(z-lib.org).pdf"),
        "This is an extra note about Clean Code principles to test multi-doc ingestion."
    ]

    client, vectorizer, df = build_and_index(
        documents=docs,
        collection=COLLECTION,
        chunk_size=300,
        overlap=50,
    )

    # Global retrieval
    query_tfidf(client, COLLECTION, vectorizer, "principios de código limpio y mantenibilidad", top_k=5)

    import pickle
    with open("vectorizer.pkl", "wb") as f:
        pickle.dump(vectorizer, f)
    print("✅ vectorizer.pkl guardado")
    # Example with a filter by source and/or page
    # query_tfidf(
    #     client, COLLECTION, vectorizer,
    #     query="naming functions and readability",
    #     top_k=5,
    #     source_filter=str(Path("./knowledge/cc_knowledge_book.pdf")),
    #     page_eq=3,
    # )

Corpus shape: (2145, 4)
Upserted batch 1/3 (1024 points)
Upserted batch 2/3 (1024 points)
Upserted batch 3/3 (97 points)

🔎 'principios de código limpio y mantenibilidad' → top 5

• 0.1543 | knowledge/Fluent.Python.2nd.Edition.(z-lib.org).pdf | p1082 | Fluent.Python.2nd.Edition.(z-lib.org).pdf:p1082_c2
  MacOS Finder window displays the flags as they are saved. The scripts are downloading images from fluentpython.com, which is behind a CDN, so you may see slower results i...

• 0.0852 | knowledge/Fluent.Python.2nd.Edition.(z-lib.org).pdf | p641 | Fluent.Python.2nd.Edition.(z-lib.org).pdf:p641_c2
  depends on the context! For such purposes as deciding how best to cook a waterfowl once you’ve bagged it, for example, specific observable traits (not all of them—plumage...

• 0.0811 | knowledge/Fluent.Python.2nd.Edition.(z-lib.org).pdf | p1083 | Fluent.Python.2nd.Edition.(z-lib.org).pdf:p1083_c1
  RU CN BR IN FR BD TR EG VN IR PH CD ET ID NG DE JP PK MX US 20 flags downloaded in 1.27s $ pyt

/var/folders/mt/ynd178zj0lsf9_dhk8rpqy200000gn/T/ipykernel_40050/4068670005.py:230: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import NamedSparseVector, SparseVector

def search_tfidf(client: QdrantClient, collection_name: str, query: str, vectorizer, top_k: int = 5):
    q = vectorizer.transform([query])
    idx = q.indices.tolist()
    vals = q.data.tolist()

    results = client.search(
        collection_name=collection_name,
        query_vector=NamedSparseVector(
            name="text",  
            vector=SparseVector(indices=idx, values=vals)
        ),
        limit=top_k,
        with_payload=True
    )

    print(f"\n🔍 Query: '{query}' → Top {top_k} results:\n")
    for r in results:
        print(f"• Score: {r.score:.4f} | Page: {r.payload.get('page')} | Chunk: {r.payload.get('chunk_id')}")
        print(f"  {r.payload.get('text', '')[:180]}...\n")

    return results

In [8]:
client = QdrantClient(url="http://localhost:6333", prefer_grpc=False)

results = search_tfidf(
    client=client,
    collection_name=COLLECTION,
    query="side efects",
    vectorizer=vectorizer,
    top_k=5
)


🔍 Query: 'side efects' → Top 5 results:

• Score: 0.1733 | Page: 75 | Chunk: cc_knowledge_book.pdf:p75_c1
  44 Chapter 3: Functions Have No Side Effects Side effects are lies. Y our function promises to do one thing, but it also does other hidden things. Sometimes it will make unexpected...

• Score: 0.1192 | Page: 328 | Chunk: Fluent.Python.2nd.Edition.(z-lib.org).pdf:p328_c1
  object created or referenced on the right-hand side. And the object must exist before a name can be bound to it, as Example 6-2 proves. Example 6-2. Variables are bound to objects ...

• Score: 0.1062 | Page: 1384 | Chunk: Fluent.Python.2nd.Edition.(z-lib.org).pdf:p1384_c1
  i. Pipelines of coroutines i. The Meaning of yield from i. Basic behavior of yield from ii. Exception handling in yield from j. Use Case: Coroutines for Discrete Event Simulation i...

• Score: 0.1048 | Page: 1061 | Chunk: Fluent.Python.2nd.Edition.(z-lib.org).pdf:p1061_c1
  Figure 20-3. One possible architecture for a data system that com

/var/folders/mt/ynd178zj0lsf9_dhk8rpqy200000gn/T/ipykernel_40050/1117325604.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
